In [ ]:
import os
import numpy as np
import pandas as pd
import scipy
import anndata
import scanpy as sc
import pybiomart
import scvi
import torch
import random
import seaborn as sns

In [ ]:
cwd = os.getcwd()
cwd

In [ ]:
meta = pd.read_csv(cwd+'/data/GSE162170_rna_cell_metadata.txt.gz', compression='gzip', sep='\t')
meta.index = meta['Cell.ID']
meta.index.name = None
meta

In [ ]:
%%time
counts = pd.read_csv(cwd+'/data/GSE162170_rna_counts.tsv.gz', compression='gzip', sep='\t')
counts = counts.transpose()
counts

In [ ]:
%%time
adata = anndata.AnnData(X=counts,
                        obs=meta,
                        var=counts.columns.to_frame())
adata

In [ ]:
%%time
a = sc.queries.biomart_annotations('hsapiens', ['ensembl_gene_id','hgnc_symbol'])
b = dict(zip(a['ensembl_gene_id'], a['hgnc_symbol']))
adata.var['hgnc_symbol'] = adata.var[0].map(b)
adata.var.index = adata.var['hgnc_symbol']
adata.var.index.name = None
adata.var.drop(columns=[0,'hgnc_symbol'], inplace=True)
adata.var

In [ ]:
adata.var_names_make_unique

In [ ]:
a = ~adata.var.index.isnull()
adata = adata[:,a].copy()

In [ ]:
adata

In [ ]:
%%time
sc.pp.filter_genes(adata, min_cells=10)
sc.pp.filter_cells(adata, min_genes=200)

In [ ]:
#sc.pp.subsample(adata, n_obs=40000, random_state=0, copy=False)

In [ ]:
%%time
adata.X = scipy.sparse.csr_matrix(adata.X.copy())
adata.layers['counts'] = scipy.sparse.csr_matrix(adata.X.copy())
#adata.layers['log2_counts'] = sc.pp.log1p(adata.layers['counts'].copy(), base=2)

In [ ]:
%%time
random.seed(17)
scvi.model.SCVI.setup_anndata(adata, layer='counts', batch_key='Sample.ID')
scvi_model = scvi.model.SCVI(adata, n_layers=2, n_latent=30, n_hidden=128, gene_likelihood='nb')
scvi_model.train()

In [ ]:
%%time
random.seed(17)
adata.obsm['X_scvi'] = scvi_model.get_latent_representation()
adata.layers['counts_scvi'] = scvi_model.get_normalized_expression(library_size=10000)
#adata.layers['log2_counts_scvi'] = sc.pp.log1p(adata.layers['counts_scvi'].copy(), base=2)

In [ ]:
%%time
sc.pp.neighbors(adata, use_rep='X_scvi', key_added='neighbors_scvi', n_neighbors=20)
sc.tl.leiden(adata, neighbors_key='neighbors_scvi', key_added='leiden_scvi', resolution=3)
sc.tl.umap(adata, neighbors_key='neighbors_scvi')
sc.pl.umap(adata, color=['leiden_scvi'], legend_loc='on data')

In [ ]:
%%time
sc.pp.neighbors(adata, use_rep='X_scvi', key_added='neighbors_scvi', n_neighbors=20)
sc.tl.leiden(adata, neighbors_key='neighbors_scvi', key_added='leiden_scvi', resolution=3)
sc.tl.umap(adata, neighbors_key='neighbors_scvi')
sc.pl.umap(adata, color=['leiden_scvi'], legend_loc='on data')

In [ ]:
%%time
adata.write(cwd+'/outs/231226_trevino_rna_scvi.h5ad')